In [1]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
import pandas as pd

In [2]:
# def getText(url):
#     res = requests.get(url)
#     html_page = res.content
#     soup = BeautifulSoup(html_page, 'html.parser')
#     #print(soup.get_text())
#     output = soup.get_text()
#     row = str(output).split('\n')
#     row = [item.strip() for item in row if not item.isspace() and item]
#     row = [item.strip() for item in row if item[0] != '<' and '//' not in item and (item[0]!='-' and item[-1]!= '-')]
#     try:
#         Findex = [row.index(item) for item in row if "Last Updated on" in item]
#         Findex = int(Findex[0])
#     except ValueError:
#         Findex = None
#     try:
#         Lindex = row.index('Other Companies')
#     except ValueError:
#         Lindex = None
#     row = row[Findex:Lindex]
#     print(row)
#     sample_dict = {"title": soup.title, 
#                   "result": row}
#     return sample_dict

In [3]:
# url = "https://csrbox.org/India_Company_Alkem-Laboratories-Limited-Maharashtra_5998"
# ans = getText(url)
# row_list = ans['result']
# url = 'https://csrbox.org/India_Company_Dish-TV-India-Limited-Uttar-Pradesh_6082'

# row = "\n".join(row_list)
# # print(row)

# updated = row.split("Implementing Partners")[0]
# row = row.split("Implementing Partners")[1]

# imp_partners = row.split("Thematic Areas")[0].replace("\n", "", 1)
# row = row.split("Thematic Areas")[1]

# thematic_areas = row.split("Sector :")[0]
# title = thematic_areas.split("\n")[-3]
# thematic_areas = row.split(title)[0].replace("\n", "", 1)
# row = row.split(title)[1]

# address = row.split("Sector : ")[0].replace("\n", "", 1)
# row = row.split("Sector : ")[1]

# sector = row.split("International Origin : ")[0]
# row = row.split("International Origin : ")[1]

# intl_origin = row.split("Company Type: ")[0].replace("\n", "", 1)
# row = row.split("Company Type: ")[1]

# company_type = row.split("CSR Financial Details (INR Cr.)",2)[0]
# row = row.split("CSR Financial Details (INR Cr.)",2)[2]

# csr_fin_details = row.split("Close",1)[0].replace("\n", "", 1)
# row = row.split("Close",1)[1]

# fin_insights = row.split("About the company")[0].replace("\n", "", 1)
# row = row.split("About the company")[1]

# about_company = row.split("CSR projects of the company")[0].replace("\n", "", 1)
# csr_projects = row.split("CSR projects of the company")[1].replace("\n", "", 1)

# # print(updated)
# # print()
# # print(imp_partners)
# # print()
# # print(thematic_areas)
# # print()
# # print(address)
# # print()
# # print(sector)
# # print()
# # print(intl_origin)
# # print()
# # print(company_type)
# # print()
# # print(csr_fin_details)
# # print()
# # print(fin_insights)
# # print()
# # print(about_company)
# # print()
# # print(csr_projects)
# # print()
# # print(csr_projects)

In [4]:
# import pandas as pd

# df = pd.DataFrame(data = [[title, address, sector, updated, imp_partners, thematic_areas, intl_origin,
#                          company_type, csr_fin_details, fin_insights, about_company, csr_projects]], 
                  
#                   columns=['title', 'address', 'sector', 'updated', 'implementation partners', 'thematic areas', 
#                            'intl origin', 'company type', 'csr finances', 'fin insights', 'about company', 'csr projects'])

# # df = pd.DataFrame(data = [address], columns=['address'])
# df

In [5]:
# projects = df["csr projects"][0].split("\n")
# projects[7]

In [6]:
# df.to_csv("sample.csv")

In [12]:
def get_scraped_df(url):
    res = requests.get(url)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    # --------------------------------------------------

    company_name = soup.h1.string

    address =  soup.find("a", {"style": "text-align:center; font-size:16px;"}).string.replace("\r", "").replace("\n", "")

    sic =  soup.find_all("p", {"id": "middle_top"})
    sector, intl_origin, company_type = sic[0].text, sic[1].text, sic[2].text
    sector, intl_origin, company_type = sector.split(": ")[1], intl_origin.split(": ")[1], company_type.split(": ")[1].replace("\r","").replace("\n","")

    lhs =  soup.find_all("ul", {"class": "lhs_ul"})
    lhs_list = []
    for i in range(len(lhs)):
        lhs_list.append(lhs[i].text)
    imp_partners = lhs_list[0]
    theme_areas = lhs_list[1]
    imp_partners = imp_partners.split("\n")
    imp_partners = [partner for partner in imp_partners if partner]
    theme_areas = theme_areas.split("\n")
    theme_areas = [area for area in theme_areas if area]
    
    about =  soup.find_all("div", {"class": "for_abt_org contnt"})
    about_content = 0
    hq = 0
    if(len(about) == 2):
        about_content = about[0].text.replace("\xa0", "").replace("\n", "")
        hq = (about[1].text).split("\n\n", 2)
        hq = hq[1].split(": ")[1].strip()

    elif(len(about) == 3):
        about_content = about[1].text.replace("\xa0", "").replace("\n", "")
        hq = (about[2].text).split("\n\n", 2)
        hq = hq[1].split(": ")[1].strip()
    content = about_content
    headquarters = hq
    projects = soup.find_all("div", {"class": "card"})
    projects = projects[0].text
    projects = projects.split("More CSR Info")
    #projects = [p.replace("\n\n", "") for p in projects]
    first_proj = projects[0].split("\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")
    first_proj = [first_proj[1] for p in projects if projects.index(p)==0]
    other_projects = [p for p in projects if projects.index(p)!=0 and projects.index(p)!=len(projects)-1]
    projects = first_proj + other_projects

    p_title = [p.replace("\n", "").split("\r", 1)[0].replace("\xa0", "").replace(">", "").strip() for p in projects]
    p_budget = [p.replace("\n", "").split("Project Budget:", 1)[1].split("\r\r\r")[0].replace("\r", "") for p in projects]
    p_desc = [p.replace("\n", "").split("Project Budget:", 1)[1].split("\r\r\r")[1].replace("\xa0", "") for p in projects]

    # Done = company_name, address, sector, intl_origin, company_type, imp_partners, theme_areas, about_content, hq
    if(len(p_title) == len(p_budget) == len(p_desc)):

        proj_df = pd.DataFrame(list(zip(p_title, p_budget, p_desc)), 
                   columns =['proj_title', 'proj_budget', 'proj_description'])
        proj_df['company_name'] = company_name
        proj_df['address'] = address
        proj_df['sector'] = sector
        proj_df['intl_origin'] = intl_origin
        proj_df['company_type'] = company_type
        proj_df['imp_partners'] = str(imp_partners)
        proj_df['theme_areas'] = str(theme_areas)
        proj_df['about_content'] = content
        proj_df['hq'] = headquarters
        proj_df = proj_df[['company_name', 'address', 'sector', 'intl_origin',
                           'company_type', 'imp_partners', 'theme_areas',
                           'about_content', 'hq', 'proj_title', 'proj_budget', 'proj_description']]

    return proj_df

In [8]:
# url = "https://csrbox.org/India_Company_Dish-TV-India-Limited-Uttar-Pradesh_6082"
# url = "https://csrbox.org/India_Company_Alkem-Laboratories-Limited-Maharashtra_5998"
# get_scraped_df(url = "https://csrbox.org/India_Company_Dish-TV-India-Limited-Uttar-Pradesh_6082")

In [16]:
company_df = pd.read_excel("input.xlsx", sheet_name = "Sheet2", skiprows=2)
links = company_df["Links"].dropna()

result = pd.DataFrame()

for link in links:
    result = result.append(get_scraped_df(url = link))

result.head(20)
# for name in (company_df["Links"].dropna())

KeyboardInterrupt: 

In [19]:
company_df = pd.read_excel("input.xlsx", sheet_name = "Sheet2", skiprows=2)
links = company_df["Links"].dropna()

result = pd.DataFrame()

for link in links:
    result = result.append(get_scraped_df(url = link), ignore_index=True)

result.to_csv("output.csv")